# SDialog dependencies

In [ ]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
import os
from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    print("Running on CoLab")

    # Installing Ollama (if you are not planning to use Ollama, you can just comment these lines to speed up the installation)
    !curl -fsSL https://ollama.com/install.sh | sh

    # Installing sdialog
    !git clone https://github.com/qanastek/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..
else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    get_ipython().system = os.system

## Local installation

Create a `.venv` using the root `requirement.txt` file and Python `3.11.14`

# Tutorial 13: Voices database

## Instanciate voices database from HuggingFace HUB

In [ ]:
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
voices_libritts = HuggingfaceVoiceDatabase("sdialog/voices-libritts")

or if you encounter any issue during the download due to timeout:

In [ ]:
%%script false --no-raise-error
!hf download sdialog/voices-libritts --repo-type dataset

If you encounter `We had to rate limit your IP (2a02:8429:4cfb:8b01:5476:95f0:3c2d:9880). To continue using our service, create a HF account or login to your existing account, and make sure you pass a HF_TOKEN if you're using the API.` please follow those steps to login (`hf auth login`) with your HuggingFace account on the huggingface cli: [URL HF CLI DOCS](https://huggingface.co/docs/huggingface_hub/guides/cli#hf-auth-login)

Once the database of voice is downloaded and created in the local cache, we will select a voice for a `20` years old `female`.

In [ ]:
voices_libritts.get_voice(gender="female", age=20)

You can also prevent voice to be selected twice, expliciting the parameter `keep_duplicate`:

In [ ]:
voices_libritts.get_voice(gender="female", age=20, keep_duplicate=False)

And when you want to reset this list of used voices you can use:

In [ ]:
voices_libritts.reset_used_voices()

## Custom local voice database

Download voices from our `demo` repository.

In [ ]:
import os

# If directory my_custom_voices is not present, download it
if os.path.exists("my_custom_voices"):
    print("my_custom_voices already exists")
else:
    !wget https://raw.githubusercontent.com/qanastek/sdialog/refs/heads/main/tests/data/my_custom_voices.zip
    !unzip my_custom_voices.zip
    !rm my_custom_voices.zip

Once the voices are downloaded in the directory `./my_custom_voices/`, we will create the metadata file that contains the ages, genders and the corresponding voice file for each of the speakers.

In [ ]:
from sdialog.audio.voice_database import LocalVoiceDatabase

With CSV metadata file:

In [ ]:
voice_database = LocalVoiceDatabase(
    directory_audios="./my_custom_voices/",
    metadata_file="./my_custom_voices/metadata.csv"
)
voice_database.get_voice(gender="female", age=20)

With TSV metadata file:

In [ ]:
voice_database = LocalVoiceDatabase(
    directory_audios="./my_custom_voices/",
    metadata_file="./my_custom_voices/metadata.tsv"
)
voice_database.get_voice(gender="female", age=21)

With JSON metadata file:

In [ ]:
voice_database = LocalVoiceDatabase(
    directory_audios="./my_custom_voices/",
    metadata_file="./my_custom_voices/metadata.json"
)

In [ ]:
voice_database.get_voice(gender="female", age=20)

# Language specific voices

By default all the voices are imported or fetch from/in the database is `english` if no language is specified.

Otherwise, you are able to mention the language you want to work with when you add or get a voice as shown in the following code snippet:

In [ ]:
voice_database = LocalVoiceDatabase(
    directory_audios="./my_custom_voices/",
    metadata_file="./my_custom_voices/metadata.json"
)

In [ ]:
voice_database.add_voice(
    gender="female",
    age=42,
    identifier="french_female_42",
    path="./my_custom_voices/french_female_42.wav",
    lang="french"
)

Now that a French voice is available in the database, we can retrieve it.

In [ ]:
voice_database.get_voice(gender="female", age=20, lang="french")

But if no voice are available in the targetted language, an error will be thrown:

In [ ]:
try:
    voice_database.get_voice(gender="female", age=20, lang="hindi")
except ValueError as e:
    print("Normal error in this case:", e)